In [3]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

## 字段参数 Field arguments

Each field type can be customized by keyword arguments.

`required`: 如果被设置为 `True`, 创建文档实例时不设置此值，将会抛出 `ValidationError` 异常。   
`default`: 默认值。  
`unique`: 整个集合中此字段唯一，否则抛出异常。bool  
`unique_with`: 整个集合中此字段和参数中的字段列表联合起来唯一，否则抛出异常。 **A field name (or list of field names)**  
`choices`: **An iterable (e.g. list, tuple or set)** of choices to which the value of this field should be
limited.  
`validation`: A callable to validate the value of the field.

### required 的使用
对于控制字段必须值很重要

In [14]:
# 单一字段的唯一
class BookBookidRequired(Document):
    bookid = StringField(required=True)
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.1")
    meta = {"db_alias": TEST_DB1}

In [15]:
BookBookidRequired(bookid="wx932qq00", name="孙子兵法").save()

<BookBookidRequired: BookBookidRequired object>

In [17]:
# 针对 required=True 的字段，如果保存时不设置值,将抛出 ValidationError  异常
bk_r = BookBookidRequired(name="庄子")

In [ ]:
bk_r.save()

### unique, unique_with 的使用
对于控制根据某几项字段形成唯一文档很重要

#### 单一字段的唯一

In [13]:
# 单一字段的唯一
class BookBookidUnique(Document):
    bookid = StringField(unique=True)
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.1")
    meta = {"db_alias": TEST_DB1}

In [6]:
BookBookidUnique(bookid="wx932qq00", name="孙子兵法").save()

<BookBookidUnique: BookBookidUnique object>

In [ ]:
BookBookidUnique(bookid="wx932qq00", name="战国策").save()
# 已经保存 bookid="wx932qq00" 时，再次执行保存此条数据，将抛出以下异常:
# NotUniqueError: Tried to save duplicate unique keys 
#(E11000 duplicate key error collection: engine_learn.book_bookid_unique index: bookid_1 dup key: { bookid: "wx932qq00" })

In [7]:
# 修改id变得唯一之后，可以正常保存
BookBookidUnique(bookid="wx932qq00aa", name="战国策").save()

<BookBookidUnique: BookBookidUnique object>

#### 联合字段的唯一

In [8]:
# 联合字段的唯一
class BookBookidUniqueWith(Document):
    bookid = StringField()
    name = StringField(unique_with=("bookid",))
    chapter = StringField(required=True, default="2.3.3.1")
    meta = {"db_alias": TEST_DB1}

In [9]:
BookBookidUniqueWith(bookid="wx932qq00", name="孙子兵法").save()

<BookBookidUniqueWith: BookBookidUniqueWith object>

In [ ]:
BookBookidUniqueWith(bookid="wx932qq00", name="孙子兵法").save()
# 不改变  bookid="wx932qq00", name="孙子兵法" 时抛出  NotUniqueError  异常
# NotUniqueError: Tried to save duplicate unique keys 
# (E11000 duplicate key error collection: engine_learn.book_bookid_unique_with index: 
# name_1_bookid_1 dup key: { name: "孙子兵法", bookid: "wx932qq00" })

In [10]:
# 不改变bookid时可以保存
BookBookidUniqueWith(bookid="wx932qq00", name="战国策").save()

<BookBookidUniqueWith: BookBookidUniqueWith object>

In [11]:
# 改变bookid,  不改变name时可以保存
BookBookidUniqueWith(bookid="wx932qq00aaa", name="孙子兵法").save()

<BookBookidUniqueWith: BookBookidUniqueWith object>

### validation  的使用

A callable to validate the value of the field. The callable takes the value as parameter and
should raise a `ValidationError` if validation fails  
这个参数对于规范字段值很有帮助，不符合筛选要求的字段值，不能被保存至数据库。

In [27]:
from mongoengine import ValidationError
def _not_empty(val):
    if not val:
        raise ValidationError('value can not be empty')

def _id_length(val):
    if len(val) != 8:
        raise ValidationError('id length is invalidate')

class BookValidate(Document):
    bookid = StringField(validation=_id_length)  # 传入可执行参数
    name = StringField(validation=_not_empty)
    chapter = StringField(required=True, default="2.3.3.1")
    meta = {"db_alias": TEST_DB1}

In [21]:
BookValidate(name="左传").save()

<BookValidate: BookValidate object>

In [ ]:
BookValidate(name="").save()
# 抛出以下异常:
# ValidationError: ValidationError (BookValidate:None) (value can not be empty: ['name'])

In [25]:
BookValidate(bookid="wx983714",name="老子").save()

<BookValidate: BookValidate object>

In [ ]:
BookValidate(bookid="wx9",name="荀子").save()
# 抛出以下异常:
# ValidationError: ValidationError (BookValidate:None) (id length is invalidate: ['bookid'])

### choices 的使用

此处演示使用复杂的 choices 元组值时的情况

Can either be nested tuples of value (stored in mongo) and a human readable key

In [30]:
SIZE = (('S', 'Small'),
        ('M', 'Medium'),
        ('L', 'Large'),
        ('XL', 'Extra Large'),
        ('XXL', 'Extra Extra Large'))
class BookSize(Document):
    size = StringField(max_length=3, choices=SIZE)
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3.3.1")
    meta = {"db_alias": TEST_DB1}

In [31]:
bookS = BookSize(bookid="wx093kid", name="春秋")

In [32]:
bookS.size = "A"
# 在保存时 抛出异常: 
# ValidationError: ValidationError (BookSize:None) (Value must be one of ['S', 'M', 'L', 'XL', 'XXL']: ['size'])

In [34]:
bookS.size = "S"

In [35]:
bookS.save()

<BookSize: BookSize object>

保存后的文档:
```bash
{
    "_id" : ObjectId("63fd727669020dc15e87533e"),
    "size" : "S",
    "bookid" : "wx093kid",
    "name" : "春秋",
    "chapter" : "2.3.3.1"
}bookS = BookSize(bookid="wx093kid", name="春秋")
```

In [ ]:
bookS2 = BookSize(bookid="wx093kid", name="春秋")
bookS2.size = "Small"
bookS2.save()
# 抛出异常
# ValidationError: ValidationError (BookSize:None) (Value must be one of ['S', 'M', 'L', 'XL', 'XXL']: ['size'])
# 即 在二维元组的情况下，也只能给此字段赋值 每个元组中 0 位置的值